In [1]:
import os
import mido
from mido import MidiFile, MidiTrack, Message, merge_tracks
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten, Bidirectional
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
#directly open a midi file
mid = MidiFile('Debussy.mid')


for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

In [22]:
#specify the path
path='data/ff7/'


notes = []
for filename in os.listdir(path):
    if filename.endswith(".mid"):
        #print(filename)
        mid = MidiFile(path + '/' + filename)
        for msg in mid:
            if not msg.is_meta and msg.channel = msg.type == 'note_on':
                data = msg.bytes()
                notes.append(data[1])

In [23]:
#normalize the notes
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

In [26]:
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30
for i in range(0, len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])

X=np.array(X)
y=np.array(y)    

# save a portion of the music for prediction
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [27]:
model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(n_prev, 1), return_sequences=True)))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./checkpoint/checkpoint_model_ff7{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', save_freq=500)

In [28]:
model.fit(X, y,
                 batch_size=32,
                 shuffle=True,
                 epochs=5,
                 callbacks=[model_save_callback],
                 verbose=1)
#fit(X, y, 20, 1, verbose=1, callbacks=[model_save_callback])

Epoch 1/5
499/974 [==============>...............] - ETA: 1:07 - loss: 0.3507
Epoch 00001: saving model to ./checkpoint/checkpoint_model_ff701.hdf5
974/974 [==============================] - 132s 136ms/step - loss: 0.3498
Epoch 2/5
 25/974 [..............................] - ETA: 2:08 - loss: 0.3644
Epoch 00002: saving model to ./checkpoint/checkpoint_model_ff702.hdf5
525/974 [===============>..............] - ETA: 1:07 - loss: 0.3493
Epoch 00002: saving model to ./checkpoint/checkpoint_model_ff702.hdf5
974/974 [==============================] - 141s 145ms/step - loss: 0.3498
Epoch 3/5
 51/974 [>.............................] - ETA: 1:51 - loss: 0.3450
Epoch 00003: saving model to ./checkpoint/checkpoint_model_ff703.hdf5
551/974 [===============>..............] - ETA: 58s - loss: 0.3490
Epoch 00003: saving model to ./checkpoint/checkpoint_model_ff703.hdf5
974/974 [==============================] - 129s 133ms/step - loss: 0.3498
Epoch 4/5
 77/974 [=>............................] - ETA: 2

In [29]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [30]:
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 110])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('results/ff7_ep5.mid')

In [ ]:
#model.save('green_model.h5')